### 0) Data Importing and Helper Function Definition:

In [1]:
import numpy as np

DATA_folder  = '../../Data/'
data = np.load(DATA_folder+'train_imgs.npy')
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')
test_lbls = np.load(DATA_folder+'test_lbls.npy')

In [2]:
data[lbls==0,:].shape

(5923, 784)

### 1) Definition of RBF Transformer Object:

RBF Transformer is a scikit-learn compatible transformer object that implements:

    - fit method       - clusters the digit-separated data, and computes cluster centers and inv. sq. deviations
    - transform method - based on obtained centers and deviations it computes the fi values (RBF layer outputs) as 
                         simple Gaussian functions -> Fi_k(x) = exp{ -sqrt( sum[ (x_i-c_k_i)/(dev_i^2) ] ) } 
                         
## Normalizing each Fi row with row sum -> STABILIZED the Regression a lot!!!

## For PCA fit_transform() method needs to be separated to fit() and then transform()!!!

#### Definition of Clustering Function:

In [3]:
### Clustering:

from sklearn.cluster import KMeans, AgglomerativeClustering

# clust = AgglomerativeClustering(n_clusters=500, linkage='complete')
# cluster_labels = clust.fit_predict(dig_data)

# Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
def form_clusters(data, n_kmeans, n_agglo):
    
    n_km = min(n_kmeans,data.shape[0])
    n_ag = min(n_agglo ,data.shape[0])
    
    lbls_set = []
    
    if n_km>0:
        kmeans = KMeans(n_clusters=n_km, random_state=0, init='k-means++', algorithm='elkan')
        lbls_kmeans = kmeans.fit_predict(data)
        lbls_set.append(lbls_kmeans)
    
    if n_ag>0:
        agglo  = AgglomerativeClustering(n_clusters=n_ag, linkage='complete')
        lbls_agglo  = agglo.fit_predict(data)
        lbls_set.append(lbls_agglo)
    
    centers     = []
    inv_sq_devs = []
    
    # Find cluster centers and covar matrix:
    for lbls in lbls_set:
        for k in range(max(lbls)+1):
            
            cluster = data[lbls==k,:]
            centers.append(cluster.mean(axis=0))
            dev = np.std(cluster,axis=0)
            dev[dev==0]+=1e-3 # to avoid Infs and NaNs
            inv_sq_devs.append(np.reciprocal(np.square(dev)))
            
        del cluster, dev
    return centers, inv_sq_devs;


#### Definition of RBF (Fi) Function:

In [4]:
# Function to compute whole Fi output for given dataset (for all RB centers)
def fi_transform(data, all_centers, all_inv_sq_devs):
    # data        - given dataset matrix for which to compute fi values
    # center      - list of all center vectors on which to compute fi vals
    # inv_sq_devs - list of all reciprocal sq. deviation vectors on which to compute fi vals     

    new_data = np.empty((data.shape[0],len(all_centers)))
    st = 0
    ch = 200
    ns = data.shape[0]
    # Process data in chunks of 200 smpls (optimal speed)
    while st<ns:
        en = min(st+ch,ns)
        for k in range(len(all_centers)):
            # new_data[st:en,k] = calc_fi_column(data[st:en], all_centers[k], all_inv_sq_devs[k])
            new_data[st:en,k] = np.dot(np.square(data[st:en] - np.repeat(all_centers[k][np.newaxis,:],data[st:en].shape[0],axis=0)),all_inv_sq_devs[k])
        st = en
    new_data = np.exp(-np.sqrt(new_data))
    return new_data

#### Definition of RBF Transformer (sklearn compatible object):

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, n_clusters=250, cl_ratio=0.5, n_fi_max=10, debug=False):
        self.n_clusters  = n_clusters             # number of clusters to be formed per digit
        self.cl_ratio    = min(1,max(0,cl_ratio)) # cluster ratio (between kMeans and Agglomerative)
        self.n_fi_max    = n_fi_max               # num of max fi vals to pick per digit
        self.centers     = []                     # list of cluster center vectors
        self.inv_sq_devs = []                     # list of cluster inv. sq. dev. vectors
        self.num_centers = []                     # list of number of centers per digit
        self.debug       = debug                  # debug flag
        if self.debug:
            print(self.n_clusters,self.cl_ratio,self.n_fi_max)
    
    # Clusters each digit and finds cluster centers and deviations:
    def fit(self, X, y):
        
        self.centers     = []
        self.inv_sq_devs = []
        self.num_centers = []
        
        # Calc. num. of clusters per digit based on assigned ratio (ratio*(num of smpls / 10))
        n_kmeans = round(self.cl_ratio*self.n_clusters)
        n_agglo  = self.n_clusters - n_kmeans
        
        if self.debug:
            print('Clustering data ',(n_kmeans,n_agglo))
        
        # Cluster the data over each digit
        for dig in range(10):
            # print('Clustering digit: ',dig)
            data = X[y==dig,:]
            centers, inv_sq_devs = form_clusters(data, n_kmeans, n_agglo)
            self.centers.extend(centers)
            self.inv_sq_devs.extend(inv_sq_devs)
            self.num_centers.append(len(centers))
        
        # Limit the n_fi_max to the min. number of centers per digit 
        self.n_fi_max = min(self.n_fi_max,min(self.num_centers))
        
        return self
    
    # Computes fi values (with Gaussian function) based on obtained cluster centers and deviations
    def transform(self, X, y=None):
        
        # Compute all Fi values:
        if self.debug:
            print('Calculating all Fi outputs !!!')
        all_fis = fi_transform(X, self.centers, self.inv_sq_devs)
        
        # Find n max fi vals over each digit:
        # print('- Selecting n max fi outputs !!!')
        n_max  = self.n_fi_max
        result = np.empty((X.shape[0],n_max*10))
        start  = 0
        for k in range(10):
            end = start + self.num_centers[k]
            result[:,(k*n_max):((k+1)*n_max)] = np.sort(all_fis[:,start:end],axis=1)[:,-n_max:]
            start = end
        del all_fis
        
        # Normalize each row with the sum of that row's elements:
        # result = np.divide(result,np.sum(result,axis=1).reshape((result.shape[0],1)))
        row_sum = np.sum(result,axis=1).reshape((result.shape[0],1))
        row_sum[row_sum==0]=1e-12
        result = np.divide(result,row_sum)
        
        if self.debug:
            print('Transform finished !!!')
        
        return result

In [6]:
# # Test:
# rbf = myRBFtransformer(n_kmeans = 10)
# a   = rbf.fit_transform(X=data[:500,:],y=lbls[:500])
# a.max(axis=1)

### 3) Pipelining the Model:

In [7]:
import types
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

n_pca      = 545 # First dimension to retain 99 % variance (stanford edu recomendation for images)!!!
n_clusters = 250
cl_ratio   = 0.5
n_fi_max   = 10

scaler = StandardScaler()
pca    = PCA(n_components=n_pca, whiten=True, random_state=1234)
rbf    = myRBFtransformer(n_clusters=n_clusters,cl_ratio=cl_ratio,n_fi_max=n_fi_max, debug=True)
logreg = LogisticRegression(tol=1e-12,C=1e+12,random_state=12,solver='liblinear')

############################################################################
# For PCA -> fit_transform() method needs to be modified so it calls fit() #
#            and transform() methods separately (not at the same time) !!! #
#                                                                          #
#         -> otherwise it will produce slightly different outputs when     #
#            transform() is called next time (comp. to fit_transform())    #
#                                                                          #
#            WHICH massively messes up with the OUTPUT of FI LAYER !!!!    #
############################################################################

def new_fit_transform(self, X, y=None, **fit_params):
    if y is None:
        # return self.fit(X, **fit_params).transform(X)
        self.fit(X, **fit_params)
        return self.transform(X)
    else:
        # return self.fit(X, y, **fit_params).transform(X)
        self.fit(X, y, **fit_params)
        return self.transform(X)
    
pca.fit_transform = types.MethodType(new_fit_transform, pca)

250 0.5 10


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pipe = Pipeline(steps=[('scal', scaler), ('pca', pca), ('rbf',rbf), ('logreg',logreg)])

In [15]:
# my_data = test_data[:10000]
# my_lbls = test_lbls[:10000]
# pipe.fit(my_data,my_lbls)
# print(pipe.score(my_data,my_lbls)*100)

In [16]:
# from sklearn.model_selection import GridSearchCV
from dask_searchcv import GridSearchCV

Ks = [50, 250, 500]
Rs = [0.15, 0.5, 0.85] # 1 kMeans, 0 - Agglo
Ns = [5, 10, 20]
Cs = [1e3, 1e6, 1e9, 1e12]

Ks = [500]
Rs = [0.0] # 1 kMeans, 0 - Agglo
Ns = [10]
Cs = [1e12]

param_grid = [
    {
        'rbf__n_clusters': Ks,
        'rbf__cl_ratio':   Rs,
        'rbf__n_fi_max':   Ns,
        'logreg__C':       Cs
    }
]


In [17]:
grid = GridSearchCV(pipe, cv=2, n_jobs=-1, param_grid=param_grid)

In [ ]:
grid.fit(data, lbls)

250 0.5 10


In [ ]:
print(grid.best_params_)

In [ ]:
print(max(grid.cv_results_['mean_test_score'])*100)

In [ ]:
print(grid.cv_results_['mean_test_score'])